In [ ]:
!pip install python-telegram-bot --upgrade
!pip install requests


In [2]:

import telegram 
import json
creds=json.loads(open('telegram.json','r').read())
print(creds)


credentials={"api_key": "569727815:AAFxFoPLHQDr1PHdZyBEOUPsyvrr0-NU0AY",
 "chat_id": "588798467"}


{'api_key': '569727815:AAFxFoPLHQDr1PHdZyBEOUPsyvrr0-NU0AY', 'chat_id': '588798467'}


In [ ]:
import telegram
import json
import os
import datetime


def notify_me(message="Job's done!"):
    filename = os.environ['HOMEPATH'] + '/telegram-works/telegram.json'
    with open(filename) as f:
        json_blob = f.read()
        credentials = json.loads(json_blob)

    # Initialize bot
    bot = telegram.Bot(token=credentials['api_key'])

    # Send message
    bot.send_message(chat_id=credentials['chat_id'], text=message)

In [ ]:
notify_me('test msg')

In [3]:
import requests
def get_updates(api_key):
    r= requests.get(f'https://api.telegram.org/bot{api_key}/getUpdates')
    print(r,r.content)
    return r
    

In [4]:
get_updates(credentials["api_key"])

<Response [200]> b'{"ok":true,"result":[{"update_id":261596973,\n"message":{"message_id":427,"from":{"id":588798467,"is_bot":false,"first_name":"Gokkulnath","last_name":"TS","username":"Gokkulnath","language_code":"en"},"chat":{"id":588798467,"first_name":"Gokkulnath","last_name":"TS","username":"Gokkulnath","type":"private"},"date":1558067576,"text":"/horoscope","entities":[{"offset":0,"length":10,"type":"bot_command"}]}}]}'


<Response [200]>

In [ ]:
#!/home/pi/py3/bin/python
import telegram 
import json
creds=json.loads(open('telegram.json','r').read())
#print(creds)
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
import logging
import datetime
import requests
from lxml import html
import subprocess

# def today_horoscope(bot, update):
#     URL='https://www.ganeshaspeaks.com/horoscopes/daily-horoscope/{}/'.format(sign)
#     daily_preds='//p[@class="margin-top-xs-0"]/text()'
#     pageContent=requests.get(URL)
#     tree = html.fromstring(pageContent.content)
#     #print(tree.xpath(daily_preds)[0].strip())
#     update.message.reply_text(tree.xpath(daily_preds)[0].strip())

def get_port80URL(bot,update):
    reply=open('/home/pi/url.txt','r').read()
    update.message.reply_text(reply)

#def execute_cmd(bot,update,args):
#    chat_id =  update.message.chat_id
#    try:
#        cmd =str(args[0])
#        host = subprocess.Popen([cmd], stdout = subprocess.PIPE).communicate()[0]
#        update.message.reply_text('Command Output {} : '.format(host))
#    except (IndexError, ValueError):
#        update.message.reply_text('Usage: /cmd  <cmd>. ')


def today_horoscope(bot, update, args):
    """Get Horoscope of the day. Pass Horoscope sign as argument.Usage: /horoscope <sign>"""
    signs=('Sagittarius', 'Taurus', 'Pisces', 'Cancer', 'Virgo', 'Capricorn', 'Aquarius', 'Libra', 'Gemini', 'Leo', 'Aries', 'Scorpio')
    chat_id = update.message.chat_id
    try:
        # args[0] should contain the time for the timer in seconds
        sign = str(args[0])
        
        if sign.title() not in signs:
            update.message.reply_text('Sorry! Please Enter a valid Horoscope Sign!. {}'.format(signs))
            return
        
        URL='https://www.ganeshaspeaks.com/horoscopes/daily-horoscope/{}/'.format(sign)
        daily_preds='//p[@class="margin-top-xs-0"]/text()'
        pageContent=requests.get(URL)
        tree = html.fromstring(pageContent.content)
        #print(tree.xpath(daily_preds)[0].strip())
        update.message.reply_text('Horoscope for {} : '.format(sign)+tree.xpath(daily_preds)[0].strip())
     
    except (IndexError, ValueError):
        update.message.reply_text('Usage: /horoscope <sign>. Valid signs {}'.format(signs))

def get_quote(bot, update):
    import random
    URL='https://www.brainyquote.com/topics/daily'
    quote='//a[@title="view quote"]/text()'
    author='//a[@title="view author"]/text()'
    pageContent=requests.get(URL)
    tree = html.fromstring(pageContent.content)
    quotes=tree.xpath(quote)
    authors=tree.xpath(author)
    rn=random.randint(0,len(quotes))
    reply = quotes[rn].strip()+' - '+authors[rn].strip()
    update.message.reply_text(reply)
    
# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)


# Define a few command handlers. These usually take the two arguments bot and
# update. Error handlers also receive the raised TelegramError object in error.
def start(bot, update):
    """Send a message when the command /start is issued."""
    update.message.reply_text('Hi!')


def help(bot, update):
    """Send a message when the command /help is issued."""
    update.message.reply_text('Help!')


def echo(bot, update):
    """Echo the user message."""
    update.message.reply_text(update.message.text)

def time(bot,update):
    """ Sends the Current Time"""
    update.message.reply_text(str(datetime.datetime.now()))

def error(bot, update, error):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, error)


def main():
    """Start the bot."""
    # Create the EventHandler and pass it your bot's token.
    updater = Updater(creds['api_key'])

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # on different commands - answer in Telegram
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("help", help))
    dp.add_handler(CommandHandler("time", time))
    
    #daily_horoscope
    dp.add_handler(CommandHandler("horoscope", today_horoscope,pass_args=True))
    dp.add_handler(CommandHandler("quote", get_quote))
    dp.add_handler(CommandHandler("url",get_port80URL ))
#    dp.add_handler(CommandHandler("cmd",execute_cmd))

    # on noncommand i.e message - echo the message on Telegram
    dp.add_handler(MessageHandler(Filters.text, echo))

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

main()
